In [1]:
# Import required packages
from os import path 
import time
# import numpy as np
import pandas as pd
import torch

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
robot = Robot(verbose=False)
J_tr, P_tr, P_ts, F = load_all_data(robot)
# P_tr_norm = normalize(P_tr, P_tr.min(axis=0), P_tr.max(axis=0))
# P_ts_norm = normalize(P_ts, P_tr.min(axis=0), P_tr.max(axis=0))

# knn = get_knn(P_tr=P_tr)
# knn = NearestNeighbors(n_neighbors=1)  
# P_tr = np.atleast_2d(P_tr)
# knn.fit(P_tr[:, :3])  
# knn = NearestNeighbors(n_neighbors=1)  
# P_tr = np.atleast_2d(P_tr_norm)
# knn.fit(P_tr_norm[:, :3])  

Building h-NNE hierarchy using FINCH...
Using PyNNDescent to compute 1st-neighbours at this step ...
Sat Sep  2 00:58:28 2023 Building RP forest with 32 trees
Sat Sep  2 00:58:47 2023 NN descent for 22 iterations
	 1  /  22
	 2  /  22
	 3  /  22
	Stopping threshold met -- exiting after 3 iterations
Step PyNNDescent done ...
Level 0: 801841 clusters
Using PyNNDescent to compute 1st-neighbours at this step ...
Sat Sep  2 01:02:14 2023 Building RP forest with 32 trees
Sat Sep  2 01:02:16 2023 NN descent for 20 iterations
	 1  /  20
	 2  /  20
	 3  /  20
	Stopping threshold met -- exiting after 3 iterations
Step PyNNDescent done ...
Level 1: 202071 clusters
Using PyNNDescent to compute 1st-neighbours at this step ...
Sat Sep  2 01:02:24 2023 Building RP forest with 26 trees
Sat Sep  2 01:02:25 2023 NN descent for 18 iterations
	 1  /  18
	 2  /  18
	 3  /  18
	Stopping threshold met -- exiting after 3 iterations
Step PyNNDescent done ...
Level 2: 49661 clusters
Using PyNNDescent to compute

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts_norm[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr_norm[neigh_idx], P_ts_norm[i]

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
neigh_idx = [2078302, 1221060]
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
i = 0
robot.position_orientation_errors_Arr_Inputs([J_tr_norm[i]], P_tr_norm[i])

In [ ]:
from pprint import pprint
def show_stat(arr):
    min = arr.min(axis=0)
    mean = arr.mean(axis=0)
    max = arr.max(axis=0)
    pprint((min, mean, max))

In [ ]:
show_stat(F)

In [ ]:
robot.joint_min, robot.joint_max

In [ ]:
J_norm = normalize(J_tr)
J_denorm = denormalize(J_norm, robot.joint_min, robot.joint_max)

In [ ]:
J_tr.min(axis=0), J_tr.max(axis=0), J_tr.mean(axis=0)

In [ ]:
J_denorm.min(axis=0), J_denorm.max(axis=0), J_denorm.mean(axis=0)

In [ ]:
J_norm.max(axis=0)

In [ ]:
df = pd.DataFrame(P_tr)
df.min().to_numpy(), df.max().to_numpy()

In [ ]:
df = pd.DataFrame(J_tr)
df.min(), df.max()

In [ ]:
df.min().to_numpy(), df.max().to_numpy()

In [ ]:
from zuko.flows import CNF, NSF, FlowModule, Unconditional
from utils.settings import config

flow = NSF(
            features=config.n,
            context=config.num_conditions,
            transforms=config.num_transforms,
            randperm=True,
            # bound=3.5,
            bins=10,
            activation=config.activation,
            hidden_features=[config.subnet_width] * config.subnet_num_layers,
        ).to(config.device)

flow

In [ ]:
train_loader = get_train_loader(J=J_tr,
                                    P=P_tr,
                                    F=F,
                                    device='cuda',
                                    batch_size=128)

# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
solver, optimizer, scheduler = get_flow_model(
    load_model=cfg.use_pretrained,
    num_transforms=7,
    subnet_width=1024,
    subnet_num_layers=3,
    lr=4.8e-4,
    lr_weight_decay=5e-3,
    decay_step_size=4e4,
    gamma=0.5,
    device='cuda')

In [ ]:
rand = np.random.randint(low=0, high=len(P_ts), size=cfg.num_eval_size)
_, position_errors, _, _ = test(
    robot=robot,
    P_ts=P_ts[rand],
    F=F,
    solver=solver,
    knn=knn,
    K=cfg.K,
    print_report=False,
)

In [ ]:
com_pos = np.zeros((100, 7))

In [ ]:
com_pos[:, :3]

In [ ]:
position_errors

In [ ]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train_cifar(config):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    # To restore a checkpoint, use `session.get_checkpoint()`.
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
           model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("my_model", exist_ok=True)
        torch.save(
            (net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report({"loss": (val_loss / val_steps), "accuracy": correct / total}, checkpoint=checkpoint)
    print("Finished Training")

In [ ]:
def _test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
def test_best_model(best_result):
    best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    print("Best trial test set accuracy: {}".format(correct / total))

In [ ]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16]),
}

In [ ]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"cpu": 2, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result)

main(num_samples=2, max_num_epochs=2, gpus_per_trial=0)

In [ ]:
print(f"#trials={len(result.trials)}")
print(f"time={time.time()-start_time}")
best_trial = result.get_best_trial("loss", "min", "all")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.metric_analysis["loss"]["min"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.metric_analysis["accuracy"]["max"]))

best_trained_model = Net(2**best_trial.config["l1"],
                         2**best_trial.config["l2"])
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)

checkpoint_value = getattr(best_trial.checkpoint, "dir_or_data", None) or best_trial.checkpoint.value
checkpoint_path = os.path.join(checkpoint_value, "checkpoint")

model_state, optimizer_state = torch.load(checkpoint_path)
best_trained_model.load_state_dict(model_state)

test_acc = _test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))